# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [6]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [7]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [8]:
import pandas as pd  # Import pandas for DataFrame handling

# Define example database tables and descriptions
data = {'table': ['customers', 'orders', 'products', 'employees'],
        'definition': ['Contains customer details (id, name, email, address)',
                       'Contains order details (order_id, customer_id, product_id, date, total_price)',
                       'Contains product details (product_id, name, category, price, stock)',
                       'Contains employee details (id, name, department, position)']}
df = pd.DataFrame(data)  # Create DataFrame for tables

# Convert tables into text format
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])
print(text_tables)


customers: Contains customer details (id, name, email, address)
orders: Contains order details (order_id, customer_id, product_id, date, total_price)
products: Contains product details (product_id, name, category, price, stock)
employees: Contains employee details (id, name, department, position)


In [9]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [10]:
print(text_tables)

customers: Contains customer details (id, name, email, address)
orders: Contains order details (order_id, customer_id, product_id, date, total_price)
products: Contains product details (product_id, name, category, price, stock)
employees: Contains employee details (id, name, department, position)


In [11]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [14]:
#Creating the prompt, with the user questions and the tables definitions.
# Example SQL Query
query_1 = "Which customers have made purchases over $500 in the last 30 days?"

# Format the prompt with table definitions and question
pqt1 = prompt_question_tables.format(tables=text_tables, question=query_1)


In [15]:
print(return_OAI(pqt1))

{
    "tables": ["customers", "orders"]
}


In [16]:
query_2 = "List all employees who work in the sales department."

# Format the prompt with table definitions and query
pqt2 = prompt_question_tables.format(tables=text_tables, question=query_2)
print(return_OAI(pqt2))


```json
{
    "tables": ["employees"]
}
```


In [17]:
query_3 = "What are the most sold products and their stock availability?"

# Format the prompt with table definitions and query
pqt3 = prompt_question_tables.format(tables=text_tables, question=query_3)
print(return_OAI(pqt3))


```json
{
    "tables": ["orders", "products"]
}
```


In [ ]:
print(return_OAI(pqt3))

```json
{
    "tables": {
        "employees": "Employee information",
        "salary": "Salary details for each year",
        "studies": "Educational studies"
    }
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [18]:
query_4 = "Calculate total revenue generated by each product category."
pqt4 = prompt_question_tables.format(tables=text_tables, question=query_4)
print(return_OAI(pqt4))


```json
{
    "tables": ["orders", "products"]
}
```


In [19]:
query_5 = "Show all orders placed by customer John Doe."
pqt5 = prompt_question_tables.format(tables=text_tables, question=query_5)
print(return_OAI(pqt5))


{
    "tables": ["customers", "orders"]
}


In [20]:
query_6 = "List employees who completed at least 10 sales this month."
pqt6 = prompt_question_tables.format(tables=text_tables, question=query_6)
print(return_OAI(pqt6))


```json
{
    "tables": ["orders", "employees"]
}
```


- GPT correctly identified necessary tables – It avoided extra tables and focused on relevant ones.
- AI handled different question types well – From sales, employees, and customer data, it adapted correctly.
- Complex queries worked well – AI understood stock levels, order history, and revenue.